# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder).

In [1]:
model_patch_size = 8
verbose          = 0
reset_kernel     = False

In [2]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(



Octave is ready <oct2py.core.Oct2Py object at 0x7f4b2cd8a140>

















In [3]:
import torch
torch.cuda.set_device(0)

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [4]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [5]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/zeroshot-moment-small-embedding:latest
input_ar: None
cpu: False


### Show configuration artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [14]:
entity = 'mi-santamaria'
project = 'deepvats'
folder = entity+'/'+project+'/'
model_family = 'zeroshot-moment'
task = 'embedding'
dataset = 'gtrends_kohls'
dataset_version = 'v2'
enc_artifact_dataset = folder + dataset + ':' + dataset_version
enc_artifact_small = folder + model_family + '-small-' + task + ':v0'
enc_artifact_base  =  folder + model_family + '-base-' + task + ':v0'
enc_artifact_large = folder + model_family + '-large-' + task + ':v0'

In [15]:
print("Getting dataset artifact: ", enc_artifact_dataset)
df_artifact = wandb_api.artifact(enc_artifact_dataset, type = 'dataset')
print("Getting small artifact: ", enc_artifact_small)
enc_artifact_small = wandb_api.artifact(enc_artifact_small, type='learner')
print("Getting base artifact: ", enc_artifact_base)
enc_artifact_base  = wandb_api.artifact(enc_artifact_base, type='learner')
print("Getting large artifact: ", enc_artifact_large)
enc_artifact_large = wandb_api.artifact(enc_artifact_large, type='learner')

Getting dataset artifact:  mi-santamaria/deepvats/gtrends_kohls:v2
Getting small artifact:  mi-santamaria/deepvats/zeroshot-moment-small-embedding:v0
Getting base artifact:  mi-santamaria/deepvats/zeroshot-moment-base-embedding:v0
Getting large artifact:  mi-santamaria/deepvats/zeroshot-moment-large-embedding:v0


In [8]:
print(df_artifact.name)
df = df_artifact.to_df()
display(df.head)
print(df.shape)

gtrends_kohls:v2


wandb:   1 of 1 files downloaded.  


<bound method NDFrame.head of               volume
2004-01-01  0.010417
2004-01-08  0.010417
2004-01-15  0.010417
2004-01-22  0.000000
2004-01-29  0.000000
...              ...
2012-05-03  0.322917
2012-05-10  0.312500
2012-05-17  0.281250
2012-05-24  0.291667
2012-05-31  0.322917

[440 rows x 1 columns]>

(440, 1)


In [18]:
print(enc_artifact_small.name)
enc_learner_small = enc_artifact_small.to_obj()

<Artifact QXJ0aWZhY3Q6MTM1Mzc4Nzc3MQ==>


wandb: Downloading large artifact zeroshot-moment-small-embedding:v0, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5


In [20]:
print(enc_artifact_base.name)
enc_learner_base  = enc_artifact_base.to_obj()

zeroshot-moment-base-embedding:v0


wandb: Downloading large artifact zeroshot-moment-base-embedding:v0, 432.97MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.3


In [21]:
print(enc_artifact_large.name)
enc_learner_large = enc_artifact_large.to_obj()

zeroshot-moment-large-embedding:v0


wandb: Downloading large artifact zeroshot-moment-large-embedding:v0, 1321.42MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:4.1


In [22]:
def count_parameters(model):
    #return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())
print(count_parameters(enc_learner_small))
print(count_parameters(enc_learner_base))
print(count_parameters(enc_learner_large))

35341512
109641608
341248520


In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())
count_parameters(enc_learner_large)

341248520

In [24]:
from fastai.losses import MSELossFlat
from dvats.encoder import MAELossFlat, EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE

In [25]:
import pandas as pd
results = pd.DataFrame(
    columns = [
        'model size',
        'n_epochs',
        'dataset_percent',
        'maskared_percent',
        'losses',
        'eval_results_pre',
        'eval_results_post',
        'time'
    ]
)

In [26]:
enc_input, _ = SlidingWindow(window_len=17, stride=2, get_y=[])(df)
enc_input.shape

(212, 1, 17)

In [27]:
vals = [5, 10, 20, 40, 80, 100]
epochs = vals[:2]
dataset_percents = [ val / 100 for val in vals ][:2]
maskared_percents = [ val / 100 for val in vals[:-1]] [:2]
sizes_percents = [5,10,20]
n_sizes = [int(np.floor(val*enc_input.shape[0]/100)) for val in sizes_percents]
n_sizes

[10, 21, 42]

In [28]:
#| export
import dvats.config as cfg_

In [29]:
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)

In [30]:
print(config['batch_size'])
print(config['r'])
print(config['analysis_mode'])

16
0.4
online


In [31]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": config['batch_size'],
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": 0.8, # Comprobando si el None es el problema
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": config['r'], #use enc_run lr,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": None,
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 5, #2.5*enc_input.shape[0]/100,
    "print_to_path": False,
    "print_path": "~/data/logs.txt",
    "print_mode": "w",
    "use_moment_masks": False,
    "mask_stateful": config['mask_stateful'],
    "mask_future": config['mask_future'],
    "mask_sync": config['mask_sync'],
    "analysis_mode": config['analysis_mode'],
    "use_wandb": config['use_wandb'],
    "norm_by_sample": config['norm_by_sample'],
    "norm_use_single_batch": config['norm_use_single_batch'],
    "show_plot": True,
    "metrics": [EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE],
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None
}

In [35]:
i = 0
for n_epochs in [5]:
    for dataset_percent in [0.3]:
        for maskared_percent in [0.4]:
            for sizes in [3]:
                print(f"--> epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                print(f" sizes {sizes}")
                result = fine_tune(
                    enc_learn           = enc_learner_small,
                    window_mask_percent = maskared_percent,
                    training_percent    = dataset_percent,
                    validation_percent  = 0.3, #1-dataset_percent if 1-dataset_percent != 0 else 0.3,
                    num_epochs          = n_epochs,
                    n_window_sizes      = sizes,
                    verbose             = 5,
                    **common_args    
                )
                results.append({
                    'model_size': "small",
                    'n_epochs': n_epochs,
                    'dataset_percent': dataset_percent,
                    'maskared_percent': maskared_percent,
                    'losses': result[0],
                    'eval_results_pre': result[1],
                    'eval_results_post': result[2],
                    'time': result[4]
                })
                print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                print(f" sizes {sizes} | time: {result[4]} -->")
                    


--> epoch 5, dataset_percent 0.3, mask 0.4
 sizes 3
[5] [ --> _get_encoder ]
[5]  [ _get_encoder ] About to exec _get_enc_input
[5] [ --> _get_enc_input ]
[5]  [ _get_enc_input ] is none enc_input? True
[5]  [ _get_enc_input ] About to get the windows
[5] [ --> windowed_dataset ]
[5]  [ _get_enc_input ] X is a DataFrame, X~(440, 1) | window_sizes 0, n_window_sizes 3
[5]  [ _get_enc_input ] X is a DataFrame | Selecting Fourier's dominant frequences
[5] [ --> Find_dominant_window_sizes_list ]
[5]  [ Find_dominant_window_sizes_list ] X ~ (440, 1)
[5]  [ Find_dominant_window_sizes_list ] Get sizes for var 0
[5] [ --> find_dominant_window_sizes_list_single ]
[5]  [ Find_dominant_window_sizes_list ] X ~ (440,)
[5]  [ Find_dominant_window_sizes_list ] Looking for - at most - the best 3 window sizes
[5]  [ Find_dominant_window_sizes_list ] Offset 0.05 max size: 22.0
[5]  [ Find_dominant_window_sizes_list ] --> Freqs
[5]  [ Find_dominant_window_sizes_list ] Find_dominant_window_sizes_list | Fre


  0% 0/7 [00:00<?, ?it/s]
 14% 1/7 [00:01<00:06,  1.06s/it]
 29% 2/7 [00:01<00:02,  1.96it/s]
 43% 3/7 [00:01<00:01,  2.94it/s]
100% 7/7 [00:01<00:00,  4.75it/s]

[5]  [ fine_tune_moment_single ] End timer -->
[5]  [ fine_tune_moment_single ] Start: 1737477460.542753 | End: 1737477462.0687225 | Duration: 1.53 seconds
[5]  [ fine_tune_moment_single ] Start timer
[5]  [ fine_tune_moment_single ] Train | wlen 17
[4] fine_tune_moment_train_ | Training loop


[4] fine_tune_moment_train | Fine tune loop | print_to_path False | batch_masks~tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
     


  0% 0/30 [00:00<?, ?it/s]

...

[4] sure_eval_moment | Trial 5 | y ~ torch.Size([4, 1, 4])
[4] Not the usual error. No padding, just fail
[4] sure_eval_moment | output <class 'NoneType'> -->
[4] fine_tune_moment_train_loop_step_ | Execution failed | Output none 
[4] fine_tune_moment_train | batch 5 ~ torch.Size([4, 1, 4]) | epoch 4 | train 9 of 30 | Loss backward | After loop step 


Traceback (most recent call last):
  File "/home/macu/work/dvats/encoder.py", line 794, in sure_eval_moment
    output = enc_learn(x_enc = y, input_mask = input_mask, mask = mask)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 227, in __call__
    return self.forward(*args, **kwargs)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 566, in forward
    return self.reconstruction(
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 299, in reconstruction
    x_enc = self.tokenizer(x=x_enc)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*a

[4] fine_tune_moment_train | -->
[5]  [ fine_tune_moment_single ] End timer -->
[5]  [ fine_tune_moment_single ] Start: 1737477468.8054008 | End: 1737477471.004766 | Duration: 2.20 seconds
[5]  [ fine_tune_moment_single ] fine_tune_moment_single | Eval Post | wlen 17
[5]  [ fine_tune_moment_single ] Start timer




  0% 0/7 [00:00<?, ?it/s]

RuntimeError: maximum size for tensor at dimension 2 is 4 but size is 8

In [ ]:
#| export
if verbose > 0: print("Execution ended")
from dvats.imports import beep
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)

In [ ]:
#| hide
if reset_kernel:
    import os
    os._exit(00)